# The Movie DataBase (TMDB)

In [1]:
import datetime
import pandas as pd
import numpy as np
import json
import re
pd.set_option('display.float_format', lambda x: '%.2f' % x) #per visualizzare i numeri senza la notazione scientifica

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.20,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500


In [3]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


1- For each movie, compute the number of cast members

In [4]:
def cast_members(data):
    item_dict = json.loads(data)
    return(len(item_dict))
credits['cast_members'] = credits['cast'].apply(cast_members)
credits[['movie_id','title','cast_members']].head()

,movie_id,title,cast_members
0,19995,Avatar,83
1,285,Pirates of the Caribbean: At World's End,34
2,206647,Spectre,83
3,49026,The Dark Knight Rises,158
4,49529,John Carter,27


2- How many movies do not have a homepage?

In [5]:
nohomepage = movies[movies['homepage'].isnull()]
conteggio = len(nohomepage.index)
print('I film che non hanno una homepage sono',conteggio)

I film che non hanno una homepage sono 3091


3- For each year, how many movies do not have a homepage?

In [6]:
movies['years'] = pd.to_datetime(movies['release_date'],format="%Y-%m-%d").dt.year   #creazione var anno
movies['years'] = movies['years'].replace(np.nan, 'Missing')
nohomepage = movies[movies['homepage'].isnull()]#selezione film senza homepage
nohomepage[['id', 'years']].groupby('years').count().tail().rename(index=str, columns={'id':'count_noHomepage'})

,count_noHomepage
years,
2013.0,127
2014.0,157
2015.0,110
2016.0,31
Missing,1


4- Extract the domain of each homepage.

In [7]:
def extract_dom(url):
    dom = []
    if (pd.isnull(url)):
        return(np.nan)
    else:
        homepage = url.split(' ')                     #per poter gestire i film con due siti: es oss 3730
        for i in range(len(homepage)):
            result=re.findall(r'[\w\-+.]+',homepage[i])
            dom.append(result[1])
        return str(dom)[1:-1].replace("'", "") 
movies['domain']=movies['homepage'].apply(extract_dom)
movies[['title', 'homepage', 'domain']][3730:3740]

,title,homepage,domain
3730,Cargo,http://www.cargoderfilm.ch http://cargothemovi...,"www.cargoderfilm.ch, cargothemovie.com"
3731,High School Musical,http://tv.disney.go.com/disneychannel/original...,tv.disney.go.com
3732,Love and Death on Long Island,NaN,NaN
3733,Night Watch,NaN,NaN
3734,The Crying Game,http://www.miramax.com/movie/the-crying-game/,www.miramax.com
3735,Porky's,NaN,NaN
3736,Survival of the Dead,http://magnetreleasing.com/survivalofthedead/,magnetreleasing.com
3737,Night of the Living Dead,NaN,NaN
3738,Lost in Translation,NaN,NaN
3739,Annie Hall,NaN,NaN


5- Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

#### Creo tabella movie_id

In [8]:
movie_id = movies[['id', 'budget', 'homepage', 'domain', 'original_language', 'original_title', 'overview', 'popularity', 'release_date','years', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count' ]]
movie_id = movie_id.rename(index=str, columns={'id': 'movie_id'})
movie_id = movie_id.set_index(['movie_id'])
movie_id.head(2)

,budget,homepage,domain,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count
movie_id,,,,,,,,,,,,,,,,
19995,237000000,http://www.avatarmovie.com/,www.avatarmovie.com,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,2009-12-10,2009.00,2787965087,162.00,Released,Enter the World of Pandora.,Avatar,7.20,11800
285,300000000,http://disney.go.com/disneypictures/pirates/,disney.go.com,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,2007-05-19,2007.00,961000000,169.00,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500


#### Creo tabella genere_id

In [9]:
genere_id = pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson = pd.read_json(movies['genres'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    genere_id = pd.concat([genere_id, tabjson])
genere_id = genere_id.rename(index=str, columns={'id': 'gender_id'})
genere = genere_id[['gender_id', 'name']]
genere_id = genere_id[['movie_id','gender_id']].set_index(['movie_id','gender_id'])
genere_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 28.0), (19995, 12.0), (19995, 14.0), (19995, 878.0), (285, 12.0)]

#### Creo tabella genere

In [10]:
genere = genere.set_index(['gender_id'])
genere = genere.drop_duplicates() #per eliminare le righe duplicate
genere.head()

,name
gender_id,
28.00,Action
12.00,Adventure
14.00,Fantasy
878.00,Science Fiction
80.00,Crime


#### Creo tabella keywords_id

In [11]:
keywords_id = pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson = pd.read_json(movies['keywords'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    keywords_id = pd.concat([keywords_id, tabjson])
keywords_id = keywords_id.rename(index=str, columns={'id': 'key_id'})
keywords = keywords_id[['key_id', 'name']]
keywords_id = keywords_id[['movie_id', 'key_id']].set_index(['movie_id', 'key_id'])
keywords_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 1463.0), (19995, 2964.0), (19995, 3386.0), (19995, 3388.0), (19995, 3679.0)]

#### Creo tabella keywords

In [12]:
keywords = keywords.set_index(['key_id'])
keywords = keywords.drop_duplicates()
keywords.head()

,name
key_id,
1463.00,culture clash
2964.00,future
3386.00,space war
3388.00,space colony
3679.00,society


#### Creo tabella prod_companies_id

In [13]:
prod_companies_id = pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson = pd.read_json(movies['production_companies'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    prod_companies_id = pd.concat([prod_companies_id, tabjson])
prod_companies_id = prod_companies_id.rename(index=str, columns={'id': 'company_id'})
prod_companies = prod_companies_id[['company_id', 'name']]
prod_companies_id = prod_companies_id[['movie_id', 'company_id']].set_index(['movie_id', 'company_id'])
prod_companies_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 289.0), (19995, 306.0), (19995, 444.0), (19995, 574.0), (285, 2.0)]

#### Creo tabella prod_companies

In [14]:
prod_companies = prod_companies.set_index(['company_id'])
prod_companies = prod_companies.drop_duplicates()
prod_companies.head()

,name
company_id,
289.00,Ingenious Film Partners
306.00,Twentieth Century Fox Film Corporation
444.00,Dune Entertainment
574.00,Lightstorm Entertainment
2.00,Walt Disney Pictures


#### Creo tabella prod_countries_id

In [15]:
prod_countries_id = pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson = pd.read_json(movies['production_countries'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    prod_countries_id = pd.concat([prod_countries_id, tabjson])
prod_countries_id = prod_countries_id.rename(index=str, columns={'iso_3166_1': 'country_id'})
prod_countries = prod_countries_id[['country_id', 'name']]
prod_countries_id = prod_countries_id[['movie_id', 'country_id']].set_index(['movie_id', 'country_id'])
prod_countries_id.head()

Empty DataFrame
Columns: []
Index: [(19995, US), (19995, GB), (285, US), (206647, GB), (206647, US)]

#### Creo tabella prod_countries

In [16]:
prod_countries = prod_countries.set_index(['country_id'])
prod_countries = prod_countries.drop_duplicates()
prod_countries.head()

,name
country_id,
US,United States of America
GB,United Kingdom
JM,Jamaica
BS,Bahamas
DM,Dominica


#### Creo tabella spoken_languages_id

In [17]:
spoken_languages_id = pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson = pd.read_json(movies['spoken_languages'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    spoken_languages_id = pd.concat([spoken_languages_id, tabjson])
spoken_languages_id = spoken_languages_id.rename(index=str, columns={'iso_639_1': 'language_id'})
spoken_languages = spoken_languages_id[['language_id', 'name']]
spoken_languages_id = spoken_languages_id[['movie_id', 'language_id']].set_index(['movie_id', 'language_id'])
spoken_languages_id.head()

Empty DataFrame
Columns: []
Index: [(19995, en), (19995, es), (285, en), (206647, fr), (206647, en)]

#### Creo tabella spoken_languages

In [18]:
spoken_languages = spoken_languages.set_index(['language_id'])
spoken_languages = spoken_languages.drop_duplicates()
spoken_languages.head()

,name
language_id,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch


#### Creo tabella credits_id

In [19]:
credits_id = credits[['movie_id', 'cast_members']]
credits_id = credits_id.set_index(['movie_id'])
credits_id.head()

,cast_members
movie_id,
19995,83
285,34
206647,83
49026,158
49529,27


#### Creo tabella cast_id

In [20]:
cast = pd.DataFrame()
for film in list(range(0,len(credits['cast']))):
    tabjson = pd.read_json(credits['cast'][film])
    idfilm = credits['movie_id'][film]
    tabjson['movie_id'] = idfilm
    cast = pd.concat([cast, tabjson])
cast = cast.rename(index=str, columns={'id': 'pers_id'})
cast_id = cast[['movie_id', 'cast_id', 'pers_id', 'order', 'credit_id', 'character']]
cast_id = cast_id.set_index(['credit_id'])
cast_id.head()

,movie_id,cast_id,pers_id,order,character
credit_id,,,,,
5602a8a7c3a3685532001c9a,19995,242.00,65731.00,0.00,Jake Sully
52fe48009251416c750ac9cb,19995,3.00,8691.00,1.00,Neytiri
52fe48009251416c750aca39,19995,25.00,10205.00,2.00,Dr. Grace Augustine
52fe48009251416c750ac9cf,19995,4.00,32747.00,3.00,Col. Quaritch
52fe48009251416c750ac9d3,19995,5.00,17647.00,4.00,Trudy Chacon


#### Creo tabella crew_id

In [21]:
crew = pd.DataFrame()
for film in list(range(0,len(credits['crew']))):
    tabjson = pd.read_json(credits['crew'][film])
    idfilm = credits['movie_id'][film]
    tabjson['movie_id'] = idfilm
    crew = pd.concat([crew, tabjson])
crew = crew.rename(index=str, columns={'id': 'pers_id'})
crew_id = crew[['movie_id', 'credit_id', 'pers_id', 'department', 'job']]
crew_id = crew_id.set_index(['credit_id'])
crew_id.head()

,movie_id,pers_id,department,job
credit_id,,,,
52fe48009251416c750aca23,19995,1721.00,Editing,Editor
539c47ecc3a36810e3001f87,19995,496.00,Art,Production Design
54491c89c3a3680fb4001cf7,19995,900.00,Sound,Sound Designer
54491cb70e0a267480001bd0,19995,900.00,Sound,Supervising Sound Editor
539c4a4cc3a36810c9002101,19995,1262.00,Production,Casting


#### Creo tabella attori

In [22]:
attori_crew = crew[['pers_id', 'gender', 'name']]
attori_cast = cast[['pers_id', 'gender', 'name']]
attori = pd.concat([attori_crew, attori_cast])
attori = attori.sort_values('pers_id').drop_duplicates().set_index('pers_id')
attori.head()

,gender,name
pers_id,,
1.00,2.00,George Lucas
2.00,2.00,Mark Hamill
3.00,2.00,Harrison Ford
4.00,1.00,Carrie Fisher
5.00,2.00,Peter Cushing


6- For each movie, compute the gross margin (difference between revenue and budget)

In [23]:
movie_id['gross_margin'] = movie_id['revenue'] - movie_id['budget']
movie_id[['title','gross_margin']].head()

,title,gross_margin
movie_id,,
19995,Avatar,2550965087
285,Pirates of the Caribbean: At World's End,661000000
206647,Spectre,635674609
49026,The Dark Knight Rises,834939099
49529,John Carter,24139100


7- For each movie, compute the number of crew members

In [24]:
members = crew[['movie_id','pers_id']].groupby('movie_id').count() #si calcolano i membri per ciascun film
#per una miglior visualizzazione dei risultati si aggiunge la colonna 'title' e si rinomina la var pers_id
members = pd.merge(members.reset_index(), movie_id.reset_index(), on='movie_id') 
members = members.rename(index=str, columns={"pers_id" : "n_CrewMembers"})
members[['movie_id', 'title', 'n_CrewMembers']].set_index('movie_id').head()

,title,n_CrewMembers
movie_id,,
5,Four Rooms,88
11,Star Wars,20
12,Finding Nemo,104
13,Forrest Gump,93
14,American Beauty,109


8- For each movie, compute the number of directors

In [25]:
directors = crew.loc[crew.job == 'Director']
num_directors = directors[['movie_id', 'pers_id']].groupby('movie_id').count()
#per una miglior visualizzazione dei risultati si aggiunge la colonna 'title' e si rinomina la var pers_id
num_directors = pd.merge(num_directors.reset_index(), movie_id.reset_index(), on='movie_id') 
num_directors = num_directors.rename(index=str, columns={'pers_id': 'n_Directors'}).head()
num_directors[['movie_id', 'title', 'n_Directors']].set_index('movie_id').head()

,title,n_Directors
movie_id,,
5,Four Rooms,4
11,Star Wars,1
12,Finding Nemo,1
13,Forrest Gump,1
14,American Beauty,1


9- For each language, compute the number of movies where such language is spoken.

In [26]:
lang = pd.merge(spoken_languages_id.reset_index(), spoken_languages.reset_index(), on = 'language_id')
lang = lang.groupby(['name', 'language_id']).count()
lang = lang.rename(index=str, columns={'movie_id': 'n_Movie'})
lang.head()

,,n_Movie
name,language_id,
,bo,4
??????,ky,1
Afrikaans,af,7
Bahasa indonesia,id,2
Bamanankan,bm,1


In [27]:
lang = lang.iloc[2:] #vengono tolte le prime due righe del dataset, non classificabili in nessuna lingua
lang.head()

,,n_Movie
name,language_id,
Afrikaans,af,7
Bahasa indonesia,id,2
Bamanankan,bm,1
Bosanski,bs,2
Català,ca,1


10- For each company and each decade, compute the overall revenue

In [28]:
#si calcola la decade
movie_id['decade'] = pd.to_datetime(movie_id['release_date'], format="%Y-%m-%d").dt.year//10
movie_id['decade'] = movie_id.decade.replace(np.nan, 'Missing')

#si calcolano il numero di compagnie per film
pc_id = prod_companies_id.reset_index()
pc_counter = pc_id.groupby('movie_id').count()  

#si prepara il data set 
pc = pd.merge(pc_counter.reset_index(), movie_id.reset_index(), on='movie_id')
pc = pc[['movie_id', 'company_id', 'revenue', 'decade']].rename(index=str, columns={'company_id':'count_companies'})

#ogni compagnia ottiene una frazone del guadagno del film
pc['revenue/companies'] = pc['revenue']/pc['count_companies']

#si visualizza il guadagno per ogni compagnia per ogni film
ALL = pd.merge(pc_id, pc[['movie_id', 'decade', 'revenue', 'count_companies', 'revenue/companies']], on='movie_id')
ALL.head()

,movie_id,company_id,decade,revenue,count_companies,revenue/companies
0,19995,289.00,200.00,2787965087,4,696991271.75
1,19995,306.00,200.00,2787965087,4,696991271.75
2,19995,444.00,200.00,2787965087,4,696991271.75
3,19995,574.00,200.00,2787965087,4,696991271.75
4,285,2.00,200.00,961000000,3,320333333.33


In [29]:
#si sommano i guadagni per ottenere il guadagno totale per ogni compagnia in una decade
ALL_grouped = ALL[['company_id', 'decade', 'revenue/companies']].groupby(['company_id','decade']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
ALL_grouped.head()

overall_revenue
company_id decade                 
1.0        197.0      434365670.17
           198.0     1187157083.17
           199.0      924317558.00
           200.0     1892716344.50
           201.0       16788459.00

11- For each decade, compute the company with maximum revenue

In [30]:
ALL_grouped2 = ALL[['company_id', 'decade','revenue/companies']].groupby(['decade', 'company_id']).sum()
ALL_grouped2 = ALL_grouped2.rename(index=str, columns={'revenue/companies':'overall_revenue'})
ALL_grouped2 = ALL_grouped2.groupby('decade').max() #per ogni decade si calcola il guadagno massimo
ALL_grouped2 = pd.merge(ALL_grouped2.reset_index(), ALL_grouped.reset_index(), on=['decade', 'overall_revenue'])  #a cui si associa la o le compagnie che hanno ottenuto tale guadagno
#per una migliore visualizzazione si associa il nome della compagnia
ALL_grouped2.head()

,decade,overall_revenue,company_id
0,191.0,4197375.50,1307.0
1,191.0,4197375.50,1308.0
2,192.0,26358000.00,8411.0
3,193.0,200088229.50,1553.0
4,193.0,200088229.50,8411.0


12- In each year, how many movies have revenue smaller than the budget?

In [31]:
movie_id1 = movie_id.reset_index()
movies_unsuccess = movie_id1.loc[movie_id1.gross_margin < 0, ['years','gross_margin']]
movies_unsuccess = movies_unsuccess.groupby('years').count()
movies_unsuccess.rename(index=str, columns={'gross_margin': 'count_unsuccess'}).tail()

,count_unsuccess
years,
2012.0,52
2013.0,62
2014.0,59
2015.0,67
2016.0,26


1- Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

In [32]:
data = pd.merge(cast_id.reset_index(), attori.reset_index(), on='pers_id')
data = pd.merge(data, credits_id.reset_index(), on='movie_id')   #preparazione data set
data = pd.merge(data, movie_id.reset_index(), on='movie_id')
data = data[['movie_id', 'pers_id', 'order', 'cast_members', 'revenue', 'title', 'name', 'character']]
data.head()

,movie_id,pers_id,order,cast_members,revenue,title,name,character
0,19995,65731.00,0.00,83,2787965087,Avatar,Sam Worthington,Jake Sully
1,19995,8691.00,1.00,83,2787965087,Avatar,Zoe Saldana,Neytiri
2,19995,10205.00,2.00,83,2787965087,Avatar,Sigourney Weaver,Dr. Grace Augustine
3,19995,32747.00,3.00,83,2787965087,Avatar,Stephen Lang,Col. Quaritch
4,19995,17647.00,4.00,83,2787965087,Avatar,Michelle Rodriguez,Trudy Chacon


In [33]:
def parti(persone, incasso, ordine):
    somma = 0
    massimo = 1
    for i in list(range(0,persone)):    #calcola le parti per cui dividere il totale
        somma = somma + 2**(i)
    parte = (incasso/somma)*(2**(persone-ordine-1))   
    return parte
data['guadagno'] = list(map(parti,data['cast_members'], data['revenue'], data['order']))
#per una migliore presentazione dei dati
guadagno = data[['movie_id', 'title', 'revenue', 'pers_id', 'name', 'character', 'order', 'guadagno']]
guadagno.head()

,movie_id,title,revenue,pers_id,name,character,order,guadagno
0,19995,Avatar,2787965087,65731.00,Sam Worthington,Jake Sully,0.00,1393982543.50
1,19995,Avatar,2787965087,8691.00,Zoe Saldana,Neytiri,1.00,696991271.75
2,19995,Avatar,2787965087,10205.00,Sigourney Weaver,Dr. Grace Augustine,2.00,348495635.88
3,19995,Avatar,2787965087,32747.00,Stephen Lang,Col. Quaritch,3.00,174247817.94
4,19995,Avatar,2787965087,17647.00,Michelle Rodriguez,Trudy Chacon,4.00,87123908.97


2- For each actor find the total revenue attributed to him/her

In [34]:
df = guadagno[['pers_id', 'guadagno']].groupby('pers_id').sum()    #si sommano incassi per attore
df = pd.merge(df.reset_index(), attori.reset_index(), on='pers_id')   #s aggiunge il nome dell'attore
df = df[['pers_id', 'name', 'guadagno']].set_index('pers_id')
df.head()

,name,guadagno
pers_id,,
1.00,George Lucas,54.87
2.00,Mark Hamill,943249261.28
3.00,Harrison Ford,2509047563.31
4.00,Carrie Fisher,248315615.29
5.00,Peter Cushing,48462375.44


3- Find the actor that is responsible for the most overall revenue

In [35]:
df.loc[df['guadagno'].idxmax()]

name          Tom Cruise
guadagno   3976119762.06
Name: 500.0, dtype: object

1- For each movie, compute the ratio between males and females in the cast

In [36]:
merged = pd.merge(attori.reset_index(), cast_id.reset_index(), on='pers_id')
merged = merged[merged.gender != 0.0] #si eliminano i valori mancanti per genere (0.0)
grouped = merged.groupby(['movie_id', 'gender']).count()
counter = grouped.reset_index()
counter = counter[['movie_id', 'gender', 'pers_id']]
counter.head()

,movie_id,gender,pers_id
0,5,1.00,14
1,5,2.00,8
2,11,1.00,3
3,11,2.00,46
4,12,1.00,5


In [37]:
femmine = counter[counter.gender == 1.0]
maschi = counter[counter.gender == 2.0]
femmine = femmine.rename(index=str, columns={"pers_id" : "femmine"})
maschi = maschi.rename(index=str, columns={"pers_id" : "maschi"})
totale = pd.merge(femmine[['movie_id', 'femmine']], maschi[['movie_id', 'maschi']], on='movie_id')
totale['ratio'] = totale['maschi']/totale['femmine']
#si aggiunge il titolo del film
totale = pd.merge(totale, movie_id.reset_index(), on='movie_id')
totale[['movie_id', 'title', 'ratio']].set_index('movie_id').head()

,title,ratio
movie_id,,
5,Four Rooms,0.57
11,Star Wars,15.33
12,Finding Nemo,3.20
13,Forrest Gump,5.00
14,American Beauty,0.68


2- For each movie, compute the ratio between the attributed revenue of males and females in the cast

In [38]:
new_table = pd.merge(guadagno, attori.reset_index(), on='pers_id')
new_table = new_table[new_table['gender'] != 0.0]
new_table = new_table.groupby(['movie_id','title', 'gender']).sum()
new_table = new_table.reset_index()[['movie_id','title', 'gender', 'guadagno']]
new_table.head()

,movie_id,title,gender,guadagno
0,5,Four Rooms,1.00,973027.53
1,5,Four Rooms,2.00,3326970.94
2,11,Star Wars,1.00,97114057.03
3,11,Star Wars,2.00,677526725.24
4,12,Finding Nemo,1.00,251728918.42


In [39]:
#si calcola il guadagno dei maschi
guadagno_maschi = new_table[new_table['gender'] == 1.0]
guadagno_maschi = guadagno_maschi[['movie_id', 'guadagno']].rename(index=str, columns={"guadagno" : "guadagno_maschi"})

#si calcola il guadagno delle femmine
guadagno_femmine = new_table[new_table['gender'] == 2.0]
guadagno_femmine = guadagno_femmine[['movie_id', 'guadagno']].rename(index=str, columns={"guadagno" : "guadagno_femmine"})

tot_guadagni = pd.merge(guadagno_maschi, guadagno_femmine, on = 'movie_id')
tot_guadagni['ratio_guadagno'] = tot_guadagni['guadagno_maschi'] / tot_guadagni['guadagno_femmine']
#si aggiunge il titolo del film per una migliore visualizzazione
tot_guadagni = pd.merge(tot_guadagni, totale, on='movie_id')
tot_guadagni = tot_guadagni[['movie_id', 'title', 'ratio_guadagno']].set_index('movie_id')
tot_guadagni.head()

,title,ratio_guadagno
movie_id,,
5,Four Rooms,0.29
11,Star Wars,0.14
12,Finding Nemo,0.37
13,Forrest Gump,0.41
14,American Beauty,0.69


3- For each director, compute the average (among all movies he/she has directed) of the ratio found in the previous point.

In [40]:
registi = crew_id[crew_id.job == 'Director']
#si ottiene il nome 
nomi_registi = pd.merge(registi.reset_index(), attori.reset_index(), on = 'pers_id')
nomi_registi = nomi_registi[['movie_id', 'pers_id', 'name']]
#si prepara il data set per le analisi
registi = pd.merge(tot_guadagni.reset_index(), nomi_registi, on='movie_id')
registi = registi.rename(index=str, columns={ 'pers_id' : 'director_id', 'name' : 'directors'})
registi.head()

,movie_id,title,ratio_guadagno,director_id,directors
0,5,Four Rooms,0.29,138.00,Quentin Tarantino
1,5,Four Rooms,0.29,2294.00,Robert Rodriguez
2,5,Four Rooms,0.29,3110.00,Allison Anders
3,5,Four Rooms,0.29,3111.00,Alexandre Rockwell
4,11,Star Wars,0.14,1.00,George Lucas


In [41]:
average_ratio = registi[['director_id', 'directors', 'ratio_guadagno']].groupby(['director_id', 'directors']).mean().reset_index()
average_ratio = average_ratio.rename(index=str, columns={ 'ratio_guadagno' : 'average_ratio'})
average_ratio.set_index('director_id').head()

,directors,average_ratio
director_id,,
1.00,George Lucas,0.20
7.00,Andrew Stanton,0.48
8.00,Lee Unkrich,0.09
13.00,Albert Brooks,nan
24.00,Robert Zemeckis,0.49


4- Find the director that has the highest average computed in the previous point

In [42]:
average_ratio.loc[average_ratio['average_ratio'].idxmax()]

director_id             66728.00
directors        Jonathan Glazer
average_ratio          524288.00
Name: 1379, dtype: object